In [1]:
import io
import sqlite3
import pandas as pd

In [2]:
!wget --quiet -P /tmp/ibd https://raw.githubusercontent.com/abnermneves/ibd/master/tp1/despesas.sql

In [3]:
conn = sqlite3.connect('despesas.db')
cursor = conn.cursor()

In [4]:
f = io.open('despesas.sql', 'r', encoding='UTF-8')
sql = f.read()
cursor.executescript(sql)

OperationalError: table [ORGAO] already exists

In [5]:
despesa = pd.read_sql_query('SELECT * FROM DESPESA_PUBLICA', conn)
orgao = pd.read_sql_query('SELECT * FROM ORGAO', conn)
programa = pd.read_sql_query('SELECT * FROM PROGRAMA_GOVERNO', conn)
funcao = pd.read_sql_query('SELECT * FROM FUNCAO', conn)
detalhamento = pd.read_sql_query('SELECT * FROM DETALHAMENTO_FUNCAO', conn)
subfuncao= pd.read_sql_query('SELECT * FROM SUBFUNCAO', conn)

<h1>Consulta 1</h1>

In [6]:
c1 = pd.read_sql_query("SELECT COD_SUBFUNCAO, NOME_SUBFUNCAO FROM DETALHAMENTO_FUNCAO NATURAL JOIN FUNCAO NATURAL JOIN SUBFUNCAO WHERE COD_FUNCAO = (SELECT CF FROM (SELECT *, MAX(QNTDSF) FROM (SELECT COD_FUNCAO AS CF, COUNT() AS QNTDSF FROM DETALHAMENTO_FUNCAO as DF GROUP BY CF)))", conn)
c1

,COD_SUBFUNCAO,NOME_SUBFUNCAO
0,125,Normatização e fiscalização
1,126,Tecnologia da informação
2,131,Comunicação social
3,244,Assistência comunitária
4,306,Alimentação e nutrição
5,365,Educação infantil
6,511,Saneamento básico rural


<h1>Consulta 6</h1>

In [7]:
c6 = pd.read_sql_query("select NOME_FUNCAO, NOME_SUBFUNCAO, SUM(VALOR_DESPESA) as VALOR_TOTAL from FUNCAO natural join DETALHAMENTO_FUNCAO natural join SUBFUNCAO natural join DESPESA_PUBLICA as DP natural join PROGRAMA_GOVERNO as PG where PG.NOME_PROGRAMA_GOVERNO = 'MAIS MEDICOS' and ANO_DESPESA = 2018 and MES_DESPESA < 10 and MES_DESPESA > 6 group by COD_FUNCAO, COD_SUBFUNCAO", conn)
c6

,NOME_FUNCAO,NOME_SUBFUNCAO,VALOR_TOTAL
0,Defesa nacional,Administração geral,0
1,Saúde,Atenção básica,11000
2,Educação,Assistência hospitalar e ambulatorial,0
3,Educação,Ensino superior,2400


<h1>Consulta 5</h1>

In [8]:
c5 = pd.read_sql_query("select COD_PROGRAMA_GOVERNO, NOME_PROGRAMA_GOVERNO from (select *, SUM(VALOR_DESPESA) as DESPESAS_TOTAIS from FUNCAO natural join DETALHAMENTO_FUNCAO natural join DESPESA_PUBLICA natural join PROGRAMA_GOVERNO where NOME_FUNCAO = 'Educação' and ANO_DESPESA = 2018 group by COD_PROGRAMA_GOVERNO order by DESPESAS_TOTAIS desc limit 3)", conn)
c5

,COD_PROGRAMA_GOVERNO,NOME_PROGRAMA_GOVERNO
0,11,ENEM
1,14,PRONATEC
2,35,MAIS MEDICOS


<h1>Consulta 7</h1>

In [9]:
c7 = pd.read_sql_query("select NOME_ORGAO, count(distinct COD_PROGRAMA_GOVERNO) as QNTD_PROGRAMAS_GOVERNO from ORGAO natural join DESPESA_PUBLICA natural join PROGRAMA_GOVERNO where VALOR_DESPESA > 0 group by COD_ORGAO", conn)
c7

,NOME_ORGAO,QNTD_PROGRAMAS_GOVERNO
0,"Ministério da Agricultura, Pecuária e Abastec",2
1,Ministério da Educação,4
2,Ministério da Saúde,2
3,Ministério da Defesa,2
4,Ministério do Desenvolvimento Social,1


<h1>Consulta 2</h1>

In [10]:
c2 = pd.read_sql_query("select NOME_FUNCAO, NOME_SUBFUNCAO, sum(VALOR_DESPESA) as DESPESAS_TOTAIS from DESPESA_PUBLICA natural join DETALHAMENTO_FUNCAO natural join FUNCAO natural join SUBFUNCAO group by COD_FUNCAO, COD_SUBFUNCAO having DESPESAS_TOTAIS > 200000", conn)
c2

,NOME_FUNCAO,NOME_SUBFUNCAO,DESPESAS_TOTAIS
0,Assistência social,Assistência comunitária,227883.77
1,Educação,Educação básica,334262.06


<h1>Consulta 8</h1>

In [11]:
c8 = pd.read_sql_query("select NOME_PROGRAMA_GOVERNO, NOME_SUBFUNCAO, count(COD_DESPESA) as QNTD_DESPESAS, sum(VALOR_DESPESA) as DESPESAS_TOTAIS from PROGRAMA_GOVERNO natural join DESPESA_PUBLICA natural join DETALHAMENTO_FUNCAO natural join SUBFUNCAO group by COD_PROGRAMA_GOVERNO, COD_SUBFUNCAO", conn)
c8

,NOME_PROGRAMA_GOVERNO,NOME_SUBFUNCAO,QNTD_DESPESAS,DESPESAS_TOTAIS
0,VIVER SEM LIMITE,Normatização e fiscalização,11,0.00
1,VIVER SEM LIMITE,Assistência ao portador de deficiência,5,0.00
2,VIVER SEM LIMITE,Assistência comunitária,19,0.00
3,VIVER SEM LIMITE,Atenção básica,18,0.00
4,VIVER SEM LIMITE,Assistência hospitalar e ambulatorial,56,0.00
5,VIVER SEM LIMITE,Ensino superior,19,0.00
6,VIVER SEM LIMITE,Educação básica,1,0.00
7,VIVER SEM LIMITE,Desenvolvimento científico,8,0.00
8,VIVER SEM LIMITE,Desenvolvimento tecnológico e engenharia,11,0.00
9,VIVER SEM LIMITE,Transferências para a educação básica,2,0.00


<h1>Consulta 12</h1>

In [12]:
c12 = pd.read_sql_query("select distinct NOME_FUNCAO, NOME_SUBFUNCAO from FUNCAO Natural join DETALHAMENTO_FUNCAO natural join SUBFUNCAO", conn)
c12

,NOME_FUNCAO,NOME_SUBFUNCAO
0,Essencial à justiça,Comunicação social
1,Defesa nacional,Administração geral
2,Defesa nacional,Defesa terrestre
3,Segurança pública,Administração geral
4,Segurança pública,Policiamento
5,Segurança pública,Defesa civil
6,Segurança pública,Informação e inteligência
7,Assistência social,Normatização e fiscalização
8,Assistência social,Tecnologia da informação
9,Assistência social,Comunicação social


<h1>Consulta 13</h1>

In [13]:
c13 = pd.read_sql_query("select distinct MODALIDADE_DESPESA, VALOR_DESPESA, NOME_PROGRAMA_GOVERNO from PROGRAMA_GOVERNO natural join (select distinct * from DESPESA_PUBLICA where MODALIDADE_DESPESA = 'Reserva de Contingência')", conn)
c13

,MODALIDADE_DESPESA,VALOR_DESPESA,NOME_PROGRAMA_GOVERNO
0,Reserva de Contingência,0.00,BRASIL SEM MISERIA
1,Reserva de Contingência,0.00,VIOLENCIA NO TRANSITO
2,Reserva de Contingência,0.00,TERRITORIOS DA CIDADANIA
3,Reserva de Contingência,8215.70,BRASIL SEM MISERIA
4,Reserva de Contingência,3.42,BRASIL SEM MISERIA
5,Reserva de Contingência,81120.13,BRASIL SEM MISERIA
6,Reserva de Contingência,0.00,REGULARIZACAO FUNDIARIA
7,Reserva de Contingência,1497.26,REGULARIZACAO FUNDIARIA
8,Reserva de Contingência,0.00,SEGURO RURAL PRIVADO
9,Reserva de Contingência,0.00,VIVER SEM LIMITE


<h1>Consulta 14</h1>

In [14]:
c14 = pd.read_sql_query("select distinct VALOR_DESPESA, NOME_ORGAO from DESPESA_PUBLICA natural join (select * from ORGAO where NOME_ORGAO = 'Ministério da Fazenda')", conn)
c14

,VALOR_DESPESA,NOME_ORGAO
0,0,Ministério da Fazenda


<h1>Consulta 15</h1>

In [15]:
c15 = pd.read_sql_query("select distinct NOME_PROGRAMA_GOVERNO, VALOR_DESPESA from DESPESA_PUBLICA natural join (select distinct * from ORGAO where NOME_ORGAO = 'Ministério da Saúde') natural join PROGRAMA_GOVERNO", conn)
c15

,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,VIVER SEM LIMITE,0.00
1,SOS EMERGENCIA,0.00
2,MELHOR EM CASA,0.00
3,UBS,0.00
4,"UPA (INVESTIMENTO, CUSTEIO E COMPLEXO DE REGUL...",0.00
5,REDE CEGONHA,0.00
6,CONTROLE DO CANCER,0.00
7,CONTROLE DO CANCER,38971.19
8,"CRACK, E POSSIVEL VENCER",0.00
9,BRASIL SEM MISERIA,0.00
